In [ ]:
%matplotlib ipympl

import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.utils import to_categorical
import pyml

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(f'{x_train.shape=} {y_train.shape=}')
print(f'{x_test.shape=} {y_test.shape=}')

idx = 10
values = x_train[idx]
number = y_train[idx]
print(f'{number=}')

ig = plt.figure
plt.imshow(values, cmap='gray')
plt.show()

In [ ]:
def preprocess_data(x, y, limit):
    all_indices = np.arange(len(x))
    all_indices = np.random.permutation(all_indices)
    all_indices = all_indices[:limit]
    x, y = x[all_indices], y[all_indices]
    x = x.reshape(len(x), 1, 28, 28)
    x = x.astype("float32") / 255
    y = to_categorical(y)

    n_cat = y.shape[1]
    y = y.reshape(len(y), n_cat, 1)
    return x, y


# training data
# load MNIST from server,
# limit to 100 images per class since we're not training on GPU
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train = preprocess_data(x_train, y_train, 5000)
x_test, y_test = preprocess_data(x_test, y_test, 1000)

print(f'{x_train.shape=} {y_train.shape=}')
print(f'{x_test.shape=} {y_test.shape=}')

In [ ]:
network = pyml.Network(pyml.BinaryCrossEntropyLoss())
network.add_layer(pyml.ConvolutionalLayer((1, 28, 28), 3, 5))
network.add_layer(pyml.ActivationLayer(pyml.SigmoidActivation()))
network.add_layer(pyml.ReshapeLayer((5, 26, 26), (5 * 26 * 26, 1)))
network.add_layer(pyml.FullyConnectedLayer(5 * 26 * 26, 100))
network.add_layer(pyml.ActivationLayer(pyml.SigmoidActivation()))
network.add_layer(pyml.FullyConnectedLayer(100, 10))
network.add_layer(pyml.ActivationLayer(pyml.SigmoidActivation()))

network.train(x_train, y_train, epochs=20, learning_rate=0.1, do_print=True)

In [ ]:
n_false = 0
for x, y in zip(x_test, y_test):
    output = network.forward(x)
    ref, pred = np.argmax(y), np.argmax(output)
    okay = ref == pred
    if not okay:
        n_false += 1


rel_false = n_false / len(x_test) * 100
print(f'Rel. false: {rel_false}%')